In [42]:
import os
import librosa, librosa.display
import matplotlib.pyplot as plt
from pathlib import Path
import tensorflow as tf
import IPython.display as ipd
import numpy as np
from scipy.fftpack import fft
import shutil
from pydub import AudioSegment

from functools import partial
#import ftransc

In [43]:
def movedir(oripath,label):
    replacestr = "\\test\\"+label+"\\"
    new_path = oripath.replace(label,replacestr)
    return  new_path
def load_audio(file_path):
    waveform, sample_rate = librosa.load(file_path)
    db = 25
    waveform_trim ,dur= librosa.effects.trim(waveform, top_db=db )
    return waveform, sample_rate

def load_audio_files(path: str, label:str):

    dataset = []
    walker = sorted(str(p) for p in Path(path).glob(f'*.wav'))
    for i, file_path in enumerate(walker):
        path, filename = os.path.split(file_path)
        path_sp = path.split("\\")
        label = path_sp[-1]
        new_path = movedir(path,label)
        
        
        if i%5 != 0:
        # Load audio
            waveform, sample_rate = load_audio(file_path)
            audio_segment = AudioSegment.from_file(file_path)
            dataset.append([waveform, sample_rate,audio_segment])
        else :
            os.makedirs(new_path, mode=0o777, exist_ok=True)
            new_path = new_path + "\\" + filename
            path = path +"\\"+filename
            shutil.copy(path,new_path)
    #     num = i
    # print(num)
    return dataset



In [44]:
#dustan_baby
burping_data = load_audio_files('./data_eval/sound_data/dustan_baby/burpme/', 'burpme')
tired_data = load_audio_files('./data_eval/sound_data/dustan_baby/tired_owh/', 'tired')
hungry_data = load_audio_files('./data_eval/sound_data/dustan_baby/hungry_Nah/', 'hungry')
poop_data = load_audio_files('./data_eval/sound_data/dustan_baby/poop_eairh/', 'poop')
discomfort_data = load_audio_files('./data_eval/sound_data/dustan_baby/uncomfortable_heh/', 'discomfort')

In [45]:
def mel_spectrogram(waveform,sample,n_fft=2560,hop_length=32):
    #fast_fourier_transf = np.fft.fft(waveform)
   
    audio_stft = librosa.feature.melspectrogram(y=waveform,n_fft=n_fft,hop_length=hop_length,fmin = 20,fmax = sample//2)
    log_spectro = librosa.power_to_db(audio_stft)
    librosa.display.specshow(log_spectro, sr=sample, x_axis='time', y_axis='mel',cmap = 'magma')
    
    
    return log_spectro

In [46]:
def mel_freq(waveform,sample):
    mfccs = librosa.feature.mfcc(waveform, sr=sample,dct_type = 2,n_mfcc=100)
    # data = librosa.power_to_db(mfccs)
    # Displaying  the MFCCs:
    librosa.display.specshow(mfccs, sr=sample, x_axis='mel',cmap = 'magma')
    return mfccs

In [47]:
def create_images(dataset,datatype, label_dir,num,real_label):
    # make directory
    mel_spectrogram_directory = f'./data_eval/img_data/{datatype}/mel_spectrogram/{label_dir}/{real_label}/'
    
    mel_spectrogram_5Class_directory = f'./data_eval/img_data/{datatype}/mel_spectrogram/allclass/{real_label}'
    
    mel_freq_directory = f'./data_eval/img_data/{datatype}/mel_freq/{label_dir}/'
    
    MFCC_5_class_directory = f'./data_eval/img_data/{datatype}/MFCC_5_class/{real_label}/'

    os.makedirs(mel_spectrogram_directory, mode=0o777, exist_ok=True)
    os.makedirs(mel_spectrogram_5Class_directory, mode=0o777, exist_ok=True)
    os.makedirs(mel_freq_directory, mode=0o777, exist_ok=True)
    os.makedirs(MFCC_5_class_directory, mode=0o777, exist_ok=True)
    for i, data in enumerate(dataset):
        sample_rate = data[1]
        waveform = data[0]
        sample_rate_cp = data[1]
        waveform_cp = data[0]
        
        plt.axis('off')    
        mel_spectrogram_spectrogram = mel_spectrogram(waveform,sample_rate)
        librosa.display.specshow(mel_spectrogram_spectrogram,sr=sample_rate, x_axis='time')
        plt.savefig(f'./data_eval/img_data/{datatype}/mel_spectrogram/{label_dir}/{real_label}/spec_img{i}{num}.jpg', bbox_inches="tight", pad_inches = 0)           
        plt.clf()
        
        plt.axis('off')    
        mel_spectrogram_spectrogram = mel_spectrogram(waveform,sample_rate)
        librosa.display.specshow(mel_spectrogram_spectrogram,sr=sample_rate, x_axis='time')
        plt.savefig(f'./data_eval/img_data/{datatype}/mel_spectrogram/allclass/{real_label}/spec_img{i}{num}.jpg', bbox_inches="tight", pad_inches = 0)           
        plt.clf()
          
        plt.axis('off')
        mel_freq_spectrogram = mel_freq(waveform_cp,sample_rate_cp)
        librosa.display.specshow(mel_freq_spectrogram,sr=sample_rate_cp, x_axis='time')
        plt.savefig(f'./data_eval/img_data/{datatype}/mel_freq/{label_dir}/spec_img{i}{num}.jpg', bbox_inches="tight", pad_inches = 0)
        plt.clf()
        
        plt.axis('off')
        mel_freq_spectrogram = mel_freq(waveform_cp,sample_rate_cp)
        librosa.display.specshow(mel_freq_spectrogram,sr=sample_rate_cp, x_axis='time')
        plt.savefig(f'./data_eval/img_data/{datatype}/MFCC_5_class/{real_label}/spec_img{i}{num}.jpg', bbox_inches="tight", pad_inches = 0)
        plt.clf()

In [48]:
create_images(burping_data,datatype = 'dustan',real_label='burping', label_dir='burpAndPoop',num=0)
create_images(poop_data,datatype = 'dustan',real_label='poop', label_dir='burpAndPoop',num=1)
create_images(hungry_data,datatype = 'dustan',real_label='hungry',label_dir= 'HungryTiredAndDiscomfort',num=2)
create_images(tired_data,datatype = 'dustan',real_label='tired',label_dir= 'HungryTiredAndDiscomfort',num=3)
create_images(discomfort_data,datatype = 'dustan',real_label='discomfort', label_dir='HungryTiredAndDiscomfort',num=4)

c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass y=[ 0.00177924 -0.00799471 -0.01225367 ... -0.00759041  0.00428651
  0.00536661] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  
c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass y=[-0.00318202 -0.00577083 -0.00303842 ...  0.0100263   0.00637814
  0.00114381] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  
c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass y=[-0.01026356  0.00125491  0.00406664 ...  0.00316731  0.00494031
  0.00305476] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  
c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass y=[-0.00206069 -0.00440482 -0.00428627 ...  0.00431337  0.00355461
  0.        ] as keyword args. From versio

<Figure size 432x288 with 0 Axes>

In [49]:
# unseen
burping_data = load_audio_files('./data_eval/sound_data/Unseen_data/burpme/', 'burpme')
tired_data = load_audio_files('./data_eval/sound_data/Unseen_data/tired_owh/', 'tired')
hungry_data = load_audio_files('./data_eval/sound_data/Unseen_data/hungry_Nah/', 'hungry')
poop_data = load_audio_files('./data_eval/sound_data/Unseen_data/poop_eairh/', 'poop')
discomfort_data = load_audio_files('./data_eval/sound_data/Unseen_data/uncomfortable_heh/', 'discomfort')

In [50]:
create_images(burping_data,datatype = 'Unseen_data',real_label='burping', label_dir='burpAndPoop',num=0)
create_images(poop_data,datatype = 'Unseen_data',real_label='poop', label_dir='burpAndPoop',num=1)
create_images(hungry_data,datatype = 'Unseen_data',real_label='hungry',label_dir= 'HungryTiredAndDiscomfort',num=2)
create_images(tired_data,datatype = 'Unseen_data',real_label='tired',label_dir= 'HungryTiredAndDiscomfort',num=3)
create_images(discomfort_data,datatype = 'Unseen_data',real_label='discomfort', label_dir='HungryTiredAndDiscomfort',num=4)

c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass y=[ 0.00039634  0.00034735  0.00015004 ... -0.00729314  0.0058878
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  
c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass y=[ 0.          0.          0.         ...  0.00931676  0.00217614
 -0.00171587] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  
c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass y=[ 0.          0.          0.         ...  0.00201074 -0.00121325
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  
c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.00090915 -0.00164769
 -0.00262179] as keyword args. From version

<Figure size 432x288 with 0 Axes>